# Quality Measure Engine (HINGE)
**Google Doc** (https://docs.google.com/document/d/1hz8OYZcb7H7SLtQxQCfPwHDus3tHoxYjuuokjQJmE1k/edit#heading=h.ar22s947w3x)
- patient ID = 647e416edb75e9b76ea46b3f
## Node types
-  **HasAValue**  : check if there is a value within the given path or not.
-  **InAList**    : check if the fetched item existed in the given list or not.
-  **OrNode**     : read several values and run logical OR.
-  **Dates**      : a special case to handle several dates.



In [100]:
# Functions

import json
from datetime import datetime
import re
import os

def TNMStaging(text):
    # Patters to match T, N, and M followed by 0 or X
    t_pattern = re.compile(r'T[0Xx]')
    n_pattern = re.compile(r'N[0Xx]')
    m_pattern = re.compile(r'M[0Xx]')

    # check for matches
    t_match = t_pattern.search(text)
    n_match = n_pattern.search(text)
    m_match = m_pattern.search(text)

    # return true only if all matches are found, otherwise return false
    return bool(t_match and n_match and m_match)

def convert_time(T):
    timestamp = int(T) / 1000  # Convert from milliseconds to seconds
    dt_object = datetime.fromtimestamp(timestamp)
    formatted_date_time = dt_object.strftime('%Y-%m-%d %H:%M:%S')
    return formatted_date_time


def checkTime(t1, t2):
    t1 = int(t1)
    t2 = int(t2)
    
    t1 /=1000
    t2 /=1000
    
    dt1 = datetime.fromtimestamp(t1)
    dt2 = datetime.fromtimestamp(t2)
    
    print(f'QMTwoDates1: {dt1}')
    print(f'QMTwoDates2: {dt2}')
    
    if dt1==dt2:
        return True
    else:
        return False
    
# def TNMStaging(text):
#     t_pattern = re.compile(r'T\d+')
#     n_pattern = re.compile(r'N\d+')
#     m_pattern = re.compile(r'M\d+')

#     t_match = t_pattern.search(text)
#     n_match = n_pattern.search(text)
#     m_match = m_pattern.search(text)

#     t_value = t_match.group() if t_match else "Not found"
#     n_value = n_match.group() if n_match else "Not found"
#     m_value = m_match.group() if m_match else "Not found"

#     print("T component:", t_value)
#     print("N component:", n_value)
#     print("M component:", m_value)

TNMStaging("cT1cNxM0")


False

In [101]:

class QMNode:

    # Read node from json and initialize required data
    # Include evaluation method, required paths, expected value(s)
    # and traversal info
    # Override with derived classes as needed.  Especially end states.

    def __init__(self, tree, nodeInfo):
        self.tree          = tree
        self.featurePath   = nodeInfo['path']
        self.successValues = nodeInfo['successValues']
        self.doEval        = nodeInfo['doEval']
        self.excludeValues = nodeInfo['excludeValues']
        self.YesBranchID   = nodeInfo['YesBranchID']
        self.NoBranchID    = nodeInfo['NoBranchID']
        # self.featureType   = nodeInfo['featureType']

    def evalExpression(self, featurePath, theData):
        
        print('\nEvaluating the following expression:\t', featurePath)

        strDate = ""
        # Handling several paths
        MetaPath = featurePath.split('/')
        if len(MetaPath)>1:
            print(MetaPath)            
            patientData = theData.getData(MetaPath[0])
            featurePath = MetaPath[1]
            strDate = "startDate"
        else:
            patientData = theData.getData('consult')
            strDate = "date"

        path = featurePath.split('.')
        for feature in path:
            if feature in patientData:
                print(feature)
                patientData = patientData.get(feature)
                if type(patientData)== list:
                    print(patientData)
                    sorted_data = sorted(patientData, key=lambda x: x[strDate], reverse=True)
                    patientData = sorted_data[0]
            else:
                return None
        return patientData

In [102]:
# OK
class QMEndNode():
    def __init__(self, tree, nodeInfo):
        self.tree = tree
        self.featurePath = ""
        self.endState = nodeInfo['featureID']

    def evaluate(self, patientData=None):
        return str(self.endState)

In [103]:
# OK
class QMHasAValue(QMNode):
    def __init__(self, tree, nodeInfo=None):
        super().__init__(tree, nodeInfo)

    def evaluate(self, patientData=None):
        
        fetchedValue = self.evalExpression(self.featurePath, patientData)
        print(f'QMHasAValue: {fetchedValue}')

        result = eval(f'{self.doEval}("{fetchedValue}")')
        print(f'QMHasAValue result: {result}')
        
        if result:
            return self.tree.getNode(self.YesBranchID).evaluate(patientData)        
        else:
            return self.tree.getNode(self.NoBranchID).evaluate(patientData)


In [104]:
# OK
class QMInAList(QMNode):
    def __init__(self, tree, nodeInfo=None):
        super().__init__(tree, nodeInfo)

    def evaluate(self, patientData=None):
        fetchedValue = self.evalExpression(self.featurePath, patientData)
        print(f'QMInAList: {fetchedValue}')

        excluding = False
        if len(self.excludeValues)>0:
            for item in self.excludeValues:
                if item in fetchedValue:
                    excluding = True
                    break

        if not excluding:
                if fetchedValue in self.successValues:
                    return self.tree.getNode(self.YesBranchID).evaluate(patientData)        
                else:
                    return self.tree.getNode(self.NoBranchID).evaluate(patientData)
        else:
            return self.tree.getNode(self.NoBranchID).evaluate(patientData)



In [105]:
# OK
class QMOrNode(QMNode):
    def __init__(self, tree, nodeInfo=None):
        super().__init__(tree, nodeInfo)

    def evaluate(self, patientData=None):
        fetchedValue = self.evalExpression(self.featurePath, patientData)
        print(f'QMOrNode: {fetchedValue}')

        result = False
        for field in self.successValues:
            print(field, fetchedValue[field])
            result = result or fetchedValue[field]
        
        if result:
            return self.tree.getNode(self.YesBranchID).evaluate(patientData)
        else:
            return self.tree.getNode(self.NoBranchID).evaluate(patientData)

In [106]:
# OK
class QMList(QMNode):
    def __init__(self, tree, nodeInfo=None):
        super().__init__(tree, nodeInfo)


    def evaluate(self, patientData=None):        
        fetchedValue = self.evalExpression(self.featurePath, patientData)
        TheList = []
        if isinstance(fetchedValue, list):
            for item in fetchedValue:
                if isinstance(item, dict) and self.doEval in item:
                    TheList.append(item[self.doEval])
        
        check = False
        for i in TheList:
            if i in self.successValues:
                check = True
                break
        if check:
            return self.tree.getNode(self.YesBranchID).evaluate(patientData)        
        else:
            return self.tree.getNode(self.NoBranchID).evaluate(patientData)

In [107]:
# OK
class QMDates(QMNode):
    '''In QMDates, 
        successVaules feature holds two values of Date1 and Date2, 
        
        Date1<Date2

        This node return True if Date1<Date2 and False otherwise.
        '''
    def __init__(self, tree, nodeInfo=None):
        super().__init__(tree, nodeInfo)

    def getDate(self, featureDate, patientData=None):
        # aim: read all values of featureDate and return the most recent one!
        pass

    def evaluate(self, patientData=None):        
        XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
        fetchedValue1 = self.evalExpression(self.successValues[0], patientData)
        fetchedValue2 = self.evalExpression(self.successValues[1], patientData)

        print(f'QMDates1: {convert_time(fetchedValue1)}')
        print(f'QMDates2: {convert_time(fetchedValue2)}')
        
        if  checkTime(fetchedValue1, fetchedValue2):
            return self.tree.getNode(self.YesBranchID).evaluate(patientData)        
        else:
            return self.tree.getNode(self.NoBranchID).evaluate(patientData)

In [108]:
# OK
class QMTree:
    dt_json = None
    path    = None
    startID = None
    
    def __init__(self, path):
        self.nodes = dict()
        self.path  = path
        self.readTreeFile()
        self.Data = None

    def readTreeFile(self):
        with open(f'{self.path}.json', 'r') as f:
            self.tree = json.load(f)
        
        self.startID = self.tree['startId']            
            
        for item in self.tree['nodes']:
            if item['type']=="List":
                self.nodes[item['featureID']] = QMList(self, item)

            elif item['type']=="HasAValue":
                self.nodes[item['featureID']] = QMHasAValue(self, item)

            elif item['type']=="InAList":
                self.nodes[item['featureID']] = QMInAList(self, item)

            elif item['type']=="QMDates":
                self.nodes[item['featureID']] = QMDates(self, item)

            elif item['type']=="OrNode":
                self.nodes[item['featureID']] = QMOrNode(self, item)

            elif item['type'] == "end" :
                self.nodes[item['featureID']] = QMEndNode(self, item)

        
    def addNode(self, node):
        pass
    
    def removeNode(self, node):
        pass
    
    def getNode(self, id):
        try:
            return self.nodes[id]
        except:
            print('Warning: no such node with id ' , id)
            return None
        
    def evaluate(self, patientData):
        self.Data = patientData
        return self.getNode(self.startID).evaluate(self.Data)
           
    def get_next(self, next_node, dt_json):
        '''search for the next item'''
        print('Next node is : ', next_node)
        for node_dict in dt_json:
            if node_dict['featureID']==next_node:
                return node_dict

In [109]:
class PatientData:
    def __init__(self, path):
        self.path = path
        temp = {f: self.read_file(f) for f in ['consult', 'eot', 'otv', 'sim']}
        self.Data = temp
        
    def getData(self, category):
        return self.Data[category]

    def read_file(self, f):
        file_path = f'{self.path}/{f}.json'
        if os.path.exists(file_path):
            with open(file_path) as file:
                return json.load(file)
        else:
            return None

In [110]:

patientData = PatientData('Data/PatientsData/patient')    

myTree = QMTree("Data/QMs/QM2")
results= myTree.evaluate(patientData)

print("Pass patients: ", results.count("pass"))
print("Fail patients: ", results.count("fail"))
print("Exclude patients: ", results.count("exclude"))




Evaluating the following expression:	 consult/boneScan.date.$date.$numberLong
['consult', 'boneScan.date.$date.$numberLong']
boneScan
date
$date
$numberLong

Evaluating the following expression:	 eot/treatmentDeliveries.startDate.$date.$numberLong
['eot', 'treatmentDeliveries.startDate.$date.$numberLong']
treatmentDeliveries
[{'id': '', 'name': 'Course 1', 'reason': 'Prostate cancer', 'intent': 'Definitive', 'modalities': 'Photons', 'techniques': 'VMAT, IMRT', 'bodySites': 'Pelvis, Prostate, SV', 'startDate': {'$date': {'$numberLong': '1477440000000'}}, 'endDate': {'$date': {'$numberLong': '1482969600000'}}, 'sessionCount': 44, 'status': 'Complete', 'uniformFractionation': True, 'volumes': [{'name': 'pelvis', 'totalDeliveredDose': 4500, 'fractionsDelivered': 25, 'uniformFractionation': True}, {'name': 'prostate+SV', 'totalDeliveredDose': 3420, 'fractionsDelivered': 19, 'uniformFractionation': True}], 'phases': [{'name': '', 'totalDeliveredDose': 0}], 'additionalInstructions': ''}]
sta